In [1]:
import re
import itertools
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("../data/raw/raw.csv")
df.head(5)

,Номер события,Номер животного,Ушная бирка животного,Пол,Номер лактации,Результат отела,Легкость отела,Дата рождения,Дней в сухостое предыдущей лактации,Дней стельности при событии,Номер группы животного,Предыдущий номер группы животного,Событие,Дни доения при событии,Дата события,Примечание события
0,0,146,598621616,F,6,FA,1,29.03.2010,62,277,18,18,РАСЧКОП,336,28.03.2019,КАН
1,1,146,598621616,F,6,FA,1,29.03.2010,62,277,18,18,ВАКЦИН,349,10.04.2019,КОГЛАВАК
2,2,146,598621616,F,6,FA,1,29.03.2010,62,277,18,18,ВАКЦИН,349,10.04.2019,ЛЕПТО
3,3,291,530073354,F,6,MA,1,30.07.2009,68,0,23,23,ПЕРЕВОД,448,08.04.2019,F001T023
4,4,291,530073354,F,6,MA,1,30.07.2009,68,0,23,23,ПРОДАНА,458,18.04.2019,ПРОДАНА ; Прочее ; УВЗ8


# Предобработка

In [3]:
col_mapper = {
    'Номер события'                      : 'event_id',
    'Пол'                                : 'sex',
    'Номер животного'                    : 'id',
    'Номер лактации'                     : 'lactation_number',
    'Результат отела'                    : 'birth_result',
    'Легкость отела'                     : 'birth_difficult',
    'Дата рождения'                      : 'birthday',
    'Дней в сухостое предыдущей лактации': 'days_no_milking',
    'Дней стельности при событии'        : 'days_pregnant',
    'Номер группы животного'             : 'group_id',
    'Предыдущий номер группы животного'  : 'prev_group_id',
    'Событие'                            : 'event',
    'Дни доения при событии'             : 'days_milking',
    'Дата события'                       : 'date',
    'Примечание события'                 : 'remark'
}

df.drop([col for col in df.columns if col not in col_mapper], axis=1, inplace=True)
df.columns = [col_mapper[col] for col in df.columns]
df.head()

,event_id,id,sex,lactation_number,birth_result,birth_difficult,birthday,days_no_milking,days_pregnant,group_id,prev_group_id,event,days_milking,date,remark
0,0,146,F,6,FA,1,29.03.2010,62,277,18,18,РАСЧКОП,336,28.03.2019,КАН
1,1,146,F,6,FA,1,29.03.2010,62,277,18,18,ВАКЦИН,349,10.04.2019,КОГЛАВАК
2,2,146,F,6,FA,1,29.03.2010,62,277,18,18,ВАКЦИН,349,10.04.2019,ЛЕПТО
3,3,291,F,6,MA,1,30.07.2009,68,0,23,23,ПЕРЕВОД,448,08.04.2019,F001T023
4,4,291,F,6,MA,1,30.07.2009,68,0,23,23,ПРОДАНА,458,18.04.2019,ПРОДАНА ; Прочее ; УВЗ8


## Проверка на пустые значения

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 592307 entries, 0 to 592306
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   event_id          592307 non-null  int64 
 1   id                592307 non-null  int64 
 2   sex               592307 non-null  object
 3   lactation_number  592307 non-null  int64 
 4   birth_result      331357 non-null  object
 5   birth_difficult   592307 non-null  int64 
 6   birthday          592307 non-null  object
 7   days_no_milking   592307 non-null  int64 
 8   days_pregnant     592307 non-null  int64 
 9   group_id          592307 non-null  int64 
 10  prev_group_id     592307 non-null  int64 
 11  event             592307 non-null  object
 12  days_milking      592307 non-null  int64 
 13  date              592307 non-null  object
 14  remark            592307 non-null  object
dtypes: int64(9), object(6)
memory usage: 67.8+ MB


В childs ставится NaN если ещё не было ребенка или животное мужского пола.
В remark стоит '-' если примечание не требуется

## Обработка событий

Привидем поле event к нормальному виду:

In [5]:
EventSynonims = {
    "ABORT"   : ["ABORT", "АБОРТ"],  
    "TOSCM"   : ["TOSCM", "НА_СХЕМУ"], 
    "NULSCM"  : ["NULSCM","СО_СХЕМЫ"],
    "DNB"     : ["DNB", "НЕОСЕМ"],
    "BRED"    : ["BRED", "ОСЕМЕН"], 
    "FRESH"   : ["FRESH", "ОТЕЛ"],  
    "PREG"    : ["PREG", "СТЕЛН"], 
    "PREGBEF" : ["PREGBEF", "СТЕЛНДО"],
    "DRY"     : ["DRY", "СУХОСТ"],
    "DRY2"    : ["DRY2", "СУХ2"],
    "OPEN"    : ["OPEN", "ЯЛОВАЯ"],
    #-------
    "WEIGHT"  : ["WEIGHT", "ВЕС"],
    "DEAD"    : ["DEAD", "ПАЛА"],
    "MOVE"    : ["MOVE", "ПЕРЕВОД"],
    "SOLD"    : ["SOLD", "ПРОДАНА"],
    #-------
    "VAC"     : ["VAC", "ВАКЦИН"],
    "VACVIR"  : ["VACVIR", "ВАКВИРУС"],
    "FOOTRIM" : ["FOOTRIM", "РАСЧКОП"],
    "HEALTH"  : ["HEALTH", "WELL", "ЗДОРОВА"],
    "BROKE"   : ["BROKE", "ДЕФЕКТ"],
    "POT"     : ["POT", "ПРОФОТ"],
    "ILLMISC" : ["ILLMISC", "БОЛЕЗНЬ"],
    "LAME"    : ["LAME", "ХРОМОТА"],
    "KETOS"   : ["KETOS", "КЕТОЗ"],
    "MAST"    : ["MAST", "МАСТИТ"],
    "METR"    : ["METR", "МЕТРИТ"],
    "PARES"   : ["PARES", "ПАРЕЗ"],
    "RP"      : ["RP", "ПОСЛЕД"],
}

EventName = dict()
for event in EventSynonims.keys():
    for label in EventSynonims[event]:
        EventName[label] = event
EventTypes = EventSynonims.keys()

Унифицируем все события к английскому:

In [6]:
df['event'] = df['event'].apply(lambda label: EventName[label])
df.head(10)

,event_id,id,sex,lactation_number,birth_result,birth_difficult,birthday,days_no_milking,days_pregnant,group_id,prev_group_id,event,days_milking,date,remark
0,0,146,F,6,FA,1,29.03.2010,62,277,18,18,FOOTRIM,336,28.03.2019,КАН
1,1,146,F,6,FA,1,29.03.2010,62,277,18,18,VAC,349,10.04.2019,КОГЛАВАК
2,2,146,F,6,FA,1,29.03.2010,62,277,18,18,VAC,349,10.04.2019,ЛЕПТО
3,3,291,F,6,MA,1,30.07.2009,68,0,23,23,MOVE,448,08.04.2019,F001T023
4,4,291,F,6,MA,1,30.07.2009,68,0,23,23,SOLD,458,18.04.2019,ПРОДАНА ; Прочее ; УВЗ8
5,5,321,F,6,FA,3,11.09.2009,62,291,18,18,MAST,213,15.03.2019,МВ
6,6,321,F,6,FA,3,11.09.2009,62,291,18,18,MAST,215,17.03.2019,ТМ6_34
7,7,321,F,6,FA,3,11.09.2009,62,291,18,18,MOVE,216,18.03.2019,F006T003
8,8,321,F,6,FA,3,11.09.2009,62,291,18,18,MAST,221,23.03.2019,КМ4_134
9,9,321,F,6,FA,3,11.09.2009,62,291,18,18,HEALTH,230,01.04.2019,МАСТИТ


Посмотрим какие есть примечания

In [7]:
def remarks(event):
    return df[df['event'] == event]['remark'].unique().tolist()
    
for event in EventTypes:
    print(event + ':\n', remarks(event))
    print('---------------------------------------------------------------------------')

ABORT:
 ['189 ДНИ', '146 ДНИ', '205 ДНИ', '94 ДНИ', '95 ДНИ', '217 ДНИ', '93 ДНИ', '167 ДНИ', '182 ДНИ', '141 ДНИ', '184 ДНИ', '51 ДНИ', '208 ДНИ', '58 ДНИ', '69 ДНИ', '155 ДНИ', '46 ДНИ', '206 ДНИ', '65 ДНИ', '200 ДНИ', '90 ДНИ', '33 ДНИ', '194 ДНИ', '56 ДНИ', '68 ДНИ', '67 ДНИ', '96 ДНИ', '59 ДНИ', '89 ДНИ', '91 ДНИ', '48 ДНИ', '55 ДНИ', '188 ДНИ', '181 ДНИ', '136 ДНИ', '214 ДНИ', '196 ДНИ', '170 ДНИ', '61 ДНИ', '104 ДНИ', '120 ДНИ', '101 ДНИ', '62 ДНИ', '76 ДНИ', '84 ДНИ', '85 ДНИ', '219 ДНИ', '229 ДНИ', '192 ДНИ', '78 ДНИ', '39 ДНИ', '60 ДНИ', '82 ДНИ', '105 ДНИ', '103 ДНИ', '216 ДНИ', '165 ДНИ', '134 ДНИ', '44 ДНИ', '110 ДНИ', '169 ДНИ', '40 ДНИ', '53 ДНИ', '215 ДНИ', '151 ДНИ', '176 ДНИ', '80 ДНИ', '108 ДНИ', '98 ДНИ', '88 ДНИ', '77 ДНИ', '198 ДНИ', '73 ДНИ', '228 ДНИ', '221 ДНИ', '140 ДНИ', '125 ДНИ', '145 ДНИ', '195 ДНИ', '123 ДНИ', '161 ДНИ', '171 ДНИ', '74 ДНИ', '47 ДНИ', '202 ДНИ', '257 ДНИ', '157 ДНИ', '106 ДНИ', '72 ДНИ', '100 ДНИ', '54 ДНИ', '81 ДНИ', '66 ДНИ', '36 ДНИ', 

VAC:
 ['КОГЛАВАК', 'ЛЕПТО', 'ТРИВАКТ', 'СУПРАВАК', 'ЛТФ', 'КРОВЬ', 'СИБИР', 'РОТАГАЛ', 'ТУБЕР', 'СКОУ']
---------------------------------------------------------------------------
VACVIR:
 ['ВИСТА']
---------------------------------------------------------------------------
FOOTRIM:
 ['КАН', 'РДК', '-', 'РДК3', 'КАН3']
---------------------------------------------------------------------------
HEALTH:
 ['МАСТИТ', 'ХРОМОТА', 'ГИНЕКОЛ', 'ПРОЧИЕ', 'МОЛОДНЯК', 'ПРОЧЕЕ', '611078', 'ХРОМАТА', '812142', 'МАТСТИТ', '912238', '102181', '22592', 'МАСТИТ,', '910092', 'ПОНОС', 'МАСТИТ3']
---------------------------------------------------------------------------
BROKE:
 ['НОГИ', '-', '2 СОСКА', 'ТУГОДОЙ', 'ATP4', 'АТР4', 'АТР1', 'СЛЕПАЯ', 'АТР2', 'АТР3', 'ВЫМЯ', 'АБСЦЕСС', 'ЖИРНАЯ', 'МАТКА', 'БРАКНОГИ', 'Т', 'БТ', 'БЕЛЬМО', '2СОСКА']
---------------------------------------------------------------------------
POT:
 ['ПРОФ2', 'ПРОФОТ']
----------------------------------------------------------------

**Механика парсинга**.


In [8]:
def splitter(s):
    return s.split(';')[1] if len(s.split(';')) > 1 else 'OTHER'

df['remark'] = df.apply(lambda row: splitter(row.remark) if (row.event == 'SOLD' or row.event == 'DEAD') else row.remark, axis=1)

In [9]:
import itertools

def rule_name(s):
    symbols = (u'АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ0123456789_', u'ABVGDEEJZIJKLMNOPRSTUFHZCSS_Y_EUA0123456789_')
    trans = {ord(a):ord(b) for a, b in zip(*symbols)}
    return ''.join(list(map(lambda val: val.translate(trans), s)))

class ParsingRule:
    def __init__(self, match, parser, name = ''):
        self.name  = rule_name(name)
        self.parse = parser
        self.match = match
    
    @staticmethod
    def match_in(match_list, parse, name=''):
        return ParsingRule(lambda x: x in match_list, parse, name)
    
    @staticmethod
    def match_str(match_str, parse, name=''):
        return ParsingRule(lambda x: x == match_str, parse, name)
    
    @staticmethod
    def match_reg(reg, parse, name=''):
        return ParsingRule(lambda x: reg.match(x), parse, name)


def satisfy(s, rules):
    return any([r.match(s) for r in rules])
    
def uniq(names):
    return [ParsingRule.match_str(s, lambda x: 1, rule_name(s)) for s in names]

def combi(rules, names):
    return rules + [ParsingRule.match_str(s, lambda x: 1, rule_name(s)) for s in names if not satisfy(s, rules)]

def comp(rules, names):
    return rules + [ParsingRule.match_in([s for s in names if not satisfy(s, rules)], lambda x: 1, rule_name('OTHER'))]

#-------

def prot_id(s):
    a = re.search(r"\d", s)
    b = re.search(r"_", s)
    m = 0 if not a else a.start()
    n = 0 if not b else b.start()
    return int(s[m:n:]) if n > m else 0

def prot_name(s):
    a = re.search(r"\d", s)
    m = 0 if not a else a.start()
    return s[0:m:] if m else s

def prot_nums(s):
    s.replace('?', '1')
    m = []
    t = s[s.find('_')+ 1::]
    for i in range(len(t)):
        if t[i] == '_' or t[i] == '-':
            for j in range(int(t[i - 1])+1, int(t[i + 1])):
                m.append(j)
        elif t[i].isdigit():
            m.append(int(t[i]))
    return set(m)


def prot(protocols):
    prot_splited = itertools.product(list(set(map(prot_name, protocols))), list(set(map(prot_id, protocols))), [0, 1, 2, 3, 4])
    return [ParsingRule(lambda x: prot_name(x) == p[0] and
                                  prot_id(x)   == p[1] and 
                                  (p[2] == 0 or p[2] in prot_nums(x)), lambda x: 1, f'{p[0]}{p[1]}_0{p[2]}') for p in prot_splited]

Задаем правила парсинга:

In [10]:
lambda_bin = lambda x: 1

# 'ABORT':
RemarksABORT = [ParsingRule.match_in(['ЖТ', 'ЖТ,'], lambda_bin, 'ЖТ'),
ParsingRule.match_in(['ДС', 'ДЛ СХЕМА', 'ДЛ СХ'], lambda_bin, 'СХ'),
ParsingRule.match_in(['БЖТ', 'БЕЗ ЖТ'], lambda_bin, 'БЖТ'),
ParsingRule.match_str('ФК', lambda_bin, 'ФК'), 
ParsingRule.match_str('ЛК', lambda_bin, 'ЛК')]

# 'HEALTH':
RemarksHEALTH = [ParsingRule.match_in(['МАТСТИТ', 'МАСТИТ,', 'МАСТИТ3', 'МАСТИТ'], lambda_bin, 'МАСТИТ'),
ParsingRule.match_in(['ХРОМОТА', 'ХРОМАТА'], lambda_bin, 'ХРОМОТА')]

#'BROKE':
RemarksBROKE = [ParsingRule.match_in(['НОГИ', 'БРАКНОГИ'], lambda_bin, 'НОГИ'),
ParsingRule.match_in(['2СОСКА', '2 СОСКА', 'ТУГОДОЙ', 'ATP4', 'АТР4', 'АТР1',  'АТР2', 'АТР3', 'ВЫМЯ'], lambda_bin,'ВЫМЯ'),
ParsingRule.match_in(['БЕЛЬМО', 'СЛЕПАЯ'], lambda_bin, 'СЛЕПАЯ')]

 #'OPEN':
RemarksOPEN = [ParsingRule.match_in(['ФК', 'ФК, СТЕЛ'], lambda_bin, 'ФК'),
ParsingRule.match_in(['ЛК', 'ЛКЭ'], lambda_bin, 'ЛК'),
ParsingRule.match_in(['ЖТ', 'ЖТ Э', 'ЖТ,', 'ЖТ3',  'ЖТМ', 'ЮЖТ', 'ЖТ,ТОЩАЯ','ЖТ ПОЛИК', 'ЖТ ДЛ СХ'], lambda_bin,'ЖТ'),
ParsingRule.match_in(['БЕЗЖТ', 'БЕЗ ЖТ'], lambda_bin,'БЖТ'),
ParsingRule.match_in(['АБС', 'БРАК АБС', 'БР МАТКА', 'БРАКАБС', 'АБ МАТКИ','ЖТ/АБС'], lambda_bin, 'АБЦ'),
ParsingRule.match_in(['ПК', 'ПОЛИКИСТ', 'ПОЛИКИС', 'ЖТ ПОЛИК'], lambda_bin,'ПК'),
ParsingRule.match_in(['ГПФ', 'ГИПОФУНК', 'ГИПОФ'], lambda_bin, 'ГПФ'),
ParsingRule.match_in(['БПАК', 'БРАК', 'ХУДАЯ', 'БРАКНОГИ', 'БРАКТОЩА', 'БРАКЖИРН', 'БР ЖИРНА', 'ИЗ БРАКА'], lambda_bin, 'БРАК'),
ParsingRule.match_in(['ФОЛ', 'ФЛ'], lambda_bin, 'ФЛ'),
ParsingRule.match_in(['ДЛ СХ', 'ДЛСХ', 'ДЛ СХЕМА', 'ДЛСХ ОХО','11 СХ','ЖТ ДЛ СХ'], lambda_bin, 'CХ')]
        
#'DNB':
RemarksDNB = [ParsingRule.match_in(['ГИНЕКОЛ' , 'ГЕНИКОЛ' , 'ГЕНЕКОЛ'], lambda_bin, 'ГИН'),
ParsingRule.match_in(['АТП' , 'АТ'], lambda_bin, 'АТ'),
ParsingRule.match_in(['БРАК МОЛ', 'МОЛОКО'], lambda_bin, 'МОЛ'),
ParsingRule.match_in(['ЖИРНАЯ', 'ЖИРН',  'ВЕС', 'ХУДАЯ', 'РОСТ', 'НЕДОРОСТ', 'РАЗВИТИЕ' , 'УЗКИЙТАЗ', 'МЕЛКАЯ'], lambda_bin, 'ВЕС'),
ParsingRule.match_in(['АБС', 'АБСЦЕСС' , 'БРАК АБС' , 'ABC' , 'АБЦ' , 'АБЦЕС' , 'АБЦ ОПУХ' , 'АБСМАТКА', 'БЕЗМАТКИ' , 'БЕЗМЕТКИ'], lambda_bin, 'АБЦ'),
ParsingRule.match_in(['АГАЛАКТ', 'АГАЛАКТИ'], lambda_bin, 'АГ'),
ParsingRule.match_in(['МАСТИТ', 'МАСТ'], lambda_bin, 'МАСТ'),
ParsingRule.match_in(['ТУГОДОЙ', 'НАДОЙ', 'ТОНК СОС',  'ВЫМЯТРОЦ' ], lambda_bin, 'ВЫМЯ'),
ParsingRule.match_in(['БРАКНОГИ', 'ТРОЦНОГИ', 'НОГИТРОЦ', 'НОГИ ТОЩ'], lambda_bin,'НОГИ'),
ParsingRule.match_in(['ТОЩ', 'ТОЩ СУСТ', 'ТОЩЬ','НОГИ ТОЩ'], lambda_bin,'ТОЩ'),
ParsingRule.match_in(['СЛЕПАЯ', 'ГЛАЗА', 'БЕЗГЛАЗА'], lambda_bin, 'CЛЕПАЯ')]
               
 #'DRY':
RemarksDRY = [ParsingRule.match_in(['CEBA', 'СЕВА', 'СEBA', 'АТР124', 'AZIT'], lambda_bin, 'СЕВА')]

In [11]:
RuleBin  = ParsingRule(lambda x: 1, lambda x: 1, 'BIN')
RuleNum  = ParsingRule.match_reg(re.compile('[0-9]*'), lambda x: int(x) if x.isdigit() else 0, 'INT') 
RuleDays = ParsingRule.match_reg(re.compile('[0-9]* ((ДНИ)|(DAYS))'), lambda x: int(x[0:-4]) if re.match('[0-9]* (ДНИ)', x) else int(x[0:-5]), 'DAYS')

EventRules = {
    'ABORT'   : comp([RuleDays] + RemarksABORT, remarks('ABORT')),
    'TOSCM'   : uniq(remarks('TOSCM')), 
    'NULSCM'  : [RuleBin],
    'DNB'     : comp(RemarksDNB, remarks('DNB')),
    'BRED'    : uniq(remarks('BRED')), 
    'FRESH'   : [RuleBin],  
    'PREG'    : [RuleDays], 
    'PREGBEF' : [RuleDays],
    'DRY'     : comp(RemarksDRY, ('DRY')),
    'DRY2'    : [RuleBin],
    'OPEN'    : comp(RemarksOPEN,('OPEN')), 
    #-------
    'WEIGHT'  : [RuleNum],
    'DEAD'    : uniq(remarks('DEAD')),
    'MOVE'    : [RuleBin],
    'SOLD'    : uniq(remarks('SOLD')),
    #-------
    'VAC'     : uniq(remarks('VAC')),
    'VACVIR'  : uniq(remarks('VACVIR')),
    'FOOTRIM' : uniq(remarks('FOOTRIM')),
    'HEALTH'  : comp(RemarksHEALTH, remarks('HEALTH')),
    'BROKE'   : comp(RemarksBROKE, remarks('BROKE')),
    'POT'     : uniq(remarks('POT')),
    'ILLMISC' : prot(remarks('ILLMISC')),
    'LAME'    : prot(remarks('LAME')),
    'KETOS'   : uniq(remarks('KETOS')),
    'MAST'    : prot(remarks('MAST')),
    'METR'    : uniq(remarks('METR')),
    'PARES'   : uniq(remarks('PARES')),
    'RP'      : uniq(remarks('RP')),
}

#Get All Rules:
print("Правила парсинга:")
for event in EventRules.keys():
    #print(event+':')
    for rule in EventRules[event]:
        rule.name = event + '_' + rule_name(rule.name.upper())
        print(rule.name)
    print('--------------------')

Правила парсинга:
ABORT_DAYS
ABORT_JT
ABORT_SH
ABORT_BJT
ABORT_FK
ABORT_LK
ABORT_OTHER
--------------------
TOSCM_5
TOSCM_9
TOSCM_11
TOSCM_2
TOSCM_13
TOSCM_14
TOSCM_15
TOSCM_16
TOSCM_40
TOSCM_31
TOSCM_29
TOSCM_30
TOSCM_38
TOSCM_33
TOSCM_46
TOSCM_34
TOSCM_36
TOSCM_55
TOSCM_60
TOSCM_53
TOSCM_37
TOSCM_52
TOSCM_42
TOSCM_43
TOSCM_58
TOSCM_47
TOSCM_48
TOSCM_41
TOSCM_49
--------------------
NULSCM_BIN
--------------------
DNB_GIN
DNB_AT
DNB_MOL
DNB_VES
DNB_ABZ
DNB_AG
DNB_MAST
DNB_VYMA
DNB_NOGI
DNB_TOS
DNB_CLEPAA
DNB_OTHER
--------------------
BRED_1H11917
BRED_1H11673
BRED_1H11978
BRED_CALGARY
BRED_1H11344
BRED_777H10778
BRED_PLUTON
BRED_777H10771
BRED_777H10777
BRED_777H10574
BRED_777H10690
BRED_1H13162
BRED_200H528
BRED_1H12902
BRED_1H12490
BRED_1H11372
BRED_777H11079
BRED_777H10915
BRED_777H11243
BRED_1H13471
BRED_200H10591
BRED_1H13411
BRED_1H11913
BRED_TOUCH
BRED_1H15057
BRED_1H13456
BRED_1H14138
BRED_777H15000
BRED_777H15074
BRED_FEIM
BRED_ANSVER
BRED_501H15074
BRED_501H15000
BRED_1H138

In [ ]:
def ext(x, rule):
    if (x.event == event and rule.match(x.remark)):
        x.rn = rule.parse(x.remark) 
    else:
        x.rn = 0
    return x

for event in EventRules.keys():
    print(event + ';')
    for rule in EventRules[event]:
        print('rule_name')
        rn = rule.name
        df = df.apply(lambda x: ext(x, rule), axis=1)

ABORT;
rule_name
rule_name
rule_name
rule_name
rule_name
rule_name
rule_name
TOSCM;
rule_name
rule_name
rule_name
rule_name
rule_name
rule_name
rule_name
rule_name
rule_name
rule_name
rule_name
rule_name
rule_name
rule_name
rule_name
rule_name
rule_name
rule_name
rule_name
rule_name
rule_name


In [ ]:
df = pd.read_csv('counters.csv', nrows=10000)

In [6]:
df[df['MAST_TM7_00'] == 0].head()

,Unnamed: 0,event_id,id,sex,lactation_number,birth_result,birth_difficult,birthday,days_no_milking,days_pregnant,...,MAST_SKM9_00,MAST_SKM9_01,MAST_SKM9_02,MAST_SKM9_03,MAST_SKM9_04,METR_E1,METR_E3,PARES_PAREZ,RP_POSLED2,RP_POSLED
5,5,5,321,F,6,FA,3,11.09.2009,62,291,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
6,6,6,321,F,6,FA,3,11.09.2009,62,291,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
8,8,8,321,F,6,FA,3,11.09.2009,62,291,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
89,89,89,1540,F,6,FA,1,18.07.2010,56,0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
92,92,92,1540,F,6,FA,1,18.07.2010,56,0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN


## Генерация оставшихся каунтеров

### Пол и дети

In [107]:
df['sex'] = df['sex'].apply(lambda x: 1 if x =='F' else 0)

In [27]:
df['childs_fa'] = df['birth_result'].apply(lambda x: str(x).count('FA'))
df['childs_ma'] = df['birth_result'].apply(lambda x: str(x).count('MA'))
df['childs_fd'] = df['birth_result'].apply(lambda x: str(x).count('FD'))
df['childs_md'] = df['birth_result'].apply(lambda x: str(x).count('MD'))

df['childs_m'] = df['birth_result'].apply(lambda x: str(x).count('M'))
df['childs_f'] = df['birth_result'].apply(lambda x: str(x).count('F'))
df['childs_d'] = df['birth_result'].apply(lambda x: str(x).count('D'))
df['childs_a'] = df['birth_result'].apply(lambda x: str(x).count('A'))


In [80]:
df=df.drop(columns=['birth_result'])
df.dtypes

event_id             int64
id                   int64
sex                  int64
lactation_number     int64
birth_difficult      int64
birthday            object
days_no_milking      int64
days_pregnant        int64
group_id             int64
prev_group_id        int64
event               object
days_milking         int64
date                object
info                object
dtype: object

**TODO** check

### Дата рождения

In [108]:
df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
df['birthday']  = pd.to_datetime(df['birthday'], infer_datetime_format=True)
df['age']  = ((df['date'] - df['birthday']) / np.timedelta64(1, 'D')).astype(int)

In [109]:
df=df.drop(columns=['birthday'])
df.dtypes

event_id                     int64
id                           int64
sex                          int64
lactation_number             int64
birth_result                object
birth_difficult              int64
days_no_milking              int64
days_pregnant                int64
group_id                     int64
prev_group_id                int64
event                       object
days_milking                 int64
date                datetime64[ns]
remark                      object
age                          int64
dtype: object

### Дата события

In [24]:
month=['jan','feb','mar','apr','may','jun', 'jul', 'aug', 'sep', 'oct', 'nov',' dec']
for m in range (12):
    df[month[m]]=df['date'].apply(lambda x: 1 if x.month == m + 1 else 0)
df.head()

,event_id,id,sex,lactation_number,birth_result,birth_difficult,days_no_milking,days_pregnant,group_id,prev_group_id,...,mar,apr,may,jun,jul,aug,sep,oct,nov,dec
0,0,146,1,6,FA,1,62,277,18,18,...,1,0,0,0,0,0,0,0,0,0
1,1,146,1,6,FA,1,62,277,18,18,...,0,1,0,0,0,0,0,0,0,0
2,2,146,1,6,FA,1,62,277,18,18,...,0,1,0,0,0,0,0,0,0,0
3,3,291,1,6,MA,1,68,0,23,23,...,0,1,0,0,0,0,0,0,0,0
4,4,291,1,6,MA,1,68,0,23,23,...,0,1,0,0,0,0,0,0,0,0


In [139]:
STATUS_TYPES = ['FRESH', 'TOSCM', 'NULSCM', 'BRED', 'OPEN', 'ABORT', 'PREG', 'PREGBEF', 'DRY', 'DRY2', 'FOOTRIM', 'DNB', 'DEAD','SOLD','BROKE']
class StatusType(IntEnum):
    #------ Осемение
    FRESH   = 1110  # Отёл
    TOSCM   = 1121  # На схему гормональной стимуляции
    NULSCM  = 1121  # Со схемы гормональной стимуляции
    BRED    = 1130  # Осеменение
    #------ Не успех
    OPEN    = 1211  # Яловая
    ABORT   = 1212  # Аборт
    #------ Успех
    PREG    = 1311  # Стельность
    PREGBEF = 1312  # Стельность с выявлением более раннего оплодотворения
    DRY     = 1321  # Сухостой (фаза 1)
    DRY2    = 1322  # Сухостой (фаза 2)
    FOOTRIM = 1330  # Обрезка копыт
    #----- Final status
    DNB     = 1411  # Выбраковка  
    DEAD    = 1412  # Смерть
    SOLD    = 1413  # Продажа
    BROKE   = 1414  # Деффект
    
    @staicmethod
    def match(label):
        global STATUS_TYPES
        return label in STATUS_TYPES


ILLNESS_TYPES = ['MAST', 'LAME', 'KETOS', 'METR', 'PARES', 'ILLMISC', 'HEALTH', 'POT', 'VAC', 'VACVIR']
class IllnessType(IntEnum):
    #----- Основные болезни
    MAST     = 2010 #
    LAME     = 2020 #
    KETOS    = 2030 #
    METR     = 2040 #
    PARES    = 2050 #
    RP       = 2060 #
    
    #------ Другие болезни    
    ILL_BUR  = 2100 # бурсит
    ILL_ABC  = 2110 # абцесс
    ILL_RUB  = 2120 # рубец
    ILL_RIN  = 2130 # рин
    ILL_BLD  = 2140 # ран
    ILL_DMG  = 2150 # повреждение ('СУСТАВ', 'УШИБ',  'ПУТЫ')
    ILL_GAST = 2160 # гастро
    ILL_PNEV = 2170 # пневмно
    ILL_LEAN = 2180 # худоба
    ILL_DIAR = 2190 # диарея 
    ILL_MICS = 2200 # остальное 'ТУГОДОЙ', 'ГИНТЕЛ', 'ТИМПАНИЯ', 'КОЛИ', 'РОТА'

    #------ События лечения
    HEALTH   = 2300 #
    POT      = 2310 #
    VAC      = 2320 #
    VACVIR   = 2330 #
    
    @staicmethod
    def match(label):
        global ILLNESS_TYPES
        return label in ILLNESS_TYPES

    
ACTION_TYPES = ['WEIGHT', 'MOVE']
class ActionType(IntEnum):
    WEIGHT   = 3310 #
    MOVE     = 3320 #
    
    @staicmethod
    def match(label, remark):
        global ACTION_TYPES
        return label in ACTION_TYPES
        
    
class EventType(IntEnum):
    STATUS  = 1 
    ILL     = 2 
    ACTION  = 3
    
    @staticmethod
    def from_str(label):
        global EventName
        event = 
        if StatusType.match(event):
            return EventType.STATUS
        elif IllnesType.match(event):
            return EventType.ILL
        elif ActionType.match(event):
            return EventType.ACTION
        return EventType(-1)


IndentationError: expected an indented block after function definition on line 29 (3635007417.py, line 31)

## Учёт истории и переводы
Теперь мы переходим к обработке контексто-зависимых забытий. Для этого надо иметь понимаение какое событие произошло ДО текущего события и ПОСЛЕ текущего события.

Как понять очередность событий, если они происходят в один день? Для начала надо посмотреть на примеры такого.

In [161]:
sf = df.groupby(['id', 'date']).count()
sf[sf['event'] > 3]

,,sex,cycle,childs,childbirth,dob,deadwood_days,pregancy_days,group_id,group_past,event,milking_days,remark
id,date,,,,,,,,,,,,
4146,7/3/2022,4,4,4,4,4,4,4,4,4,4,4,4
4245,7/29/2022,4,4,4,4,4,4,4,4,4,4,4,4
6214,7/1/2022,4,4,4,4,4,4,4,4,4,4,4,4
8036,7/29/2022,4,4,4,4,4,4,4,4,4,4,4,4
11293,7/20/2022,4,4,0,4,4,4,4,4,4,4,4,4
12171,7/20/2022,4,4,0,4,4,4,4,4,4,4,4,4
20398,7/21/2022,4,4,0,4,4,4,4,4,4,4,4,4
20855,7/12/2022,4,4,0,4,4,4,4,4,4,4,4,4
803063,7/8/2022,4,4,4,4,4,4,4,4,4,4,4,4


Иногда (но очень редко), с коровой происходит несколько событий в один день. Как правило эти события отделены переводом, а между переводами порядок не важен.

Кстати параметр "Предыдущая группа животного" указывает на группу в предыдущем событии! Сортировка событий в течении дня происходит так:
1. Разбить события на на группы (каждая группа отвечает что делали с животным когда оно было в соотвествующей группе)
2. Упорядочить группы в соответсвии с информацией о перводах
3. Упорядочить события внутри группы по порядку из Enuma

### События перемещения:
Надо проверить, что события перемещения отображают корректную информацию

In [111]:
df['incorrect_move'] = df.apply(lambda row: 1 if row.event == EventType.MOVE and int(row.remark[1:4]) != row.group_id else 0, axis=1)
df[df['incorrect_move'] == 1].head()

,id,sid,sex,cycle,childs,childbirth,dob,deadwood_days,pregancy_days,group_id,group_past,event,milking_days,date,remark,incorrect_move


### Сортировка событий:

**TODO**:
Написано максимально криво надо подправить. К тому же есть вопросы к эффективности

In [9]:
print(df[df['event'] == EventType.FRESH]['group_id'].unique().tolist())

cond = ((df['event'] == EventType.FRESH) & (df['group_id'] == 18))
df[cond].head(10)

[18, 17, 45, 33, 40]


,id,sid,sex,cycle,childs,childbirth,dob,deadwood_days,pregancy_days,group_id,group_past,event,milking_days,date,remark,event_index
126,8136,8136,F,1,FA,1,8/17/2020,0,0,18,8,106,0,7/1/2022,-,127
353,612062,612062,F,4,MA,1,12/12/2016,63,0,18,21,106,0,7/1/2022,-,354
432,811180,811180,F,3,MA,1,11/24/2018,57,0,18,10,106,0,7/1/2022,-,433
475,907168,907168,F,2,FA,1,7/24/2019,64,0,18,4,106,0,7/1/2022,-,476
480,907225,907225,F,2,FA,1,7/30/2019,71,0,18,10,106,0,7/1/2022,-,481
487,908049,908049,F,2,MA,1,8/9/2019,49,0,18,43,106,0,7/1/2022,-,488
502,909158,909158,F,2,FA,1,9/21/2019,57,0,18,10,106,0,7/1/2022,-,503
512,910131,910131,F,2,FA,1,10/19/2019,57,0,18,10,106,0,7/1/2022,-,513
514,910171,910171,F,2,FA,1,10/26/2019,64,0,18,10,106,0,7/1/2022,-,515
604,807075,807075,F,3,FA,1,7/14/2018,58,0,18,2,106,0,7/2/2022,-,605


In [12]:
import itertools
def path_sort(edge_list):
    m = len(edge_list)
    if m == 1:
        return edge_list
    
    for sigma in itertools.permutations(list(range(m))):
        bad=False
        for i in range(1, m):
            if edge_list[sigma[i]][1] != edge_list[sigma[i-1]][0]:
                bad=True
                break
        if not bad:
            edge_list_fixed = [edge_list[sigma[i]] for i in range(1, m)]
            return edge_list_fixed
    return edge_list


def get_day_ordering(r):
    
    cow_id   = r.id
    ev_date  = r.date 
    ev_index = r.event_index
    
    global df
    today = df[(df['id'] == cow_id) & (df['date'] == ev_date)].copy()
    
    moves = today[today['event'] == EventType.MOVE]
    if moves.shape[0] == 0:
        today.sort_values(by=['event'])
        today['ord'] = list(range(1, today.shape[0] + 1, 1))
        return (today[today['event_index'] == ev_index]['ord'].iloc[0])
   
    moves['to']   = moves['remark'].apply(lambda x: int(x[5::]))
    moves['from'] = moves['remark'].apply(lambda x: int(x[1:4:]))
    
    edge_list  = path_sort(moves.apply(lambda row: (row['from'], row['to']), axis=1).tolist())
    path = [edge[0] for edge in edge_list]
    path.append(edge_list[-1][1])
    
    try:
        today['group_ord'] = today['group_id'].apply(lambda x: path.index(x))
        today.sort_values(by=['group_ord', 'event'])
    except:
        #print('------------------')
        #print(edge_list)
        #print(row)
        #print('------------------')
        today.sort_values(by=['event'])
        
    today['ord'] = list(range(1, today.shape[0] + 1, 1))
    
    return (today[today['event_index'] == ev_index]['ord'].iloc[0])

In [13]:
df['ord'] = df.apply(lambda row: get_day_ordering(row), axis=1)
df.sort_values(by=['id', 'date', 'ord'])

------------------
[(44, 21)]
id                    8136
sid                   8136
sex                      F
cycle                    1
childs                  FA
childbirth               1
dob              8/17/2020
deadwood_days            0
pregancy_days            0
group_id                44
group_past               8
event                  303
milking_days             0
date              7/1/2022
remark            F044T021
event_index            126
ord                      1
Name: 125, dtype: object
------------------
------------------
[(44, 21)]
id                    8136
sid                   8136
sex                      F
cycle                    1
childs                  FA
childbirth               1
dob              8/17/2020
deadwood_days            0
pregancy_days            0
group_id                18
group_past               8
event                  106
milking_days             0
date              7/1/2022
remark                   -
event_index            127
ord  

------------------
[(5, 12)]
id                 702036
sid                702036
sex                     F
cycle                   4
childs                 FA
childbirth              1
dob              2/5/2017
deadwood_days          52
pregancy_days         158
group_id                1
group_past             12
event                 207
milking_days          205
date             7/5/2022
remark             СУСТАВ
event_index          1533
ord                     1
Name: 1532, dtype: object
------------------
------------------
[(5, 12)]
id                 702036
sid                702036
sex                     F
cycle                   4
childs                 FA
childbirth              1
dob              2/5/2017
deadwood_days          52
pregancy_days         158
group_id               12
group_past             12
event                 204
milking_days          205
date             7/5/2022
remark            ХРОМОТА
event_index          1534
ord                     2
Name: 1533, d

------------------
[(17, 45)]
id                  709076
sid                 709076
sex                      F
cycle                    3
childs                  MA
childbirth               1
dob              9/16/2017
deadwood_days           63
pregancy_days          215
group_id                42
group_past              17
event                  109
milking_days           343
date              7/6/2022
remark                   -
event_index           1869
ord                      1
Name: 1868, dtype: object
------------------
------------------
[(17, 45)]
id                  709076
sid                 709076
sex                      F
cycle                    3
childs                  MA
childbirth               1
dob              9/16/2017
deadwood_days           63
pregancy_days          215
group_id                17
group_past              17
event                  303
milking_days           343
date              7/6/2022
remark            F017T045
event_index           1870
ord 

------------------
[(6, 12)]
id                    8036
sid                   8036
sex                      F
cycle                    1
childs                  FD
childbirth               2
dob               8/6/2020
deadwood_days            0
pregancy_days            0
group_id                 6
group_past               8
event                  105
milking_days            59
date             7/29/2022
remark           614H15481
event_index           4489
ord                      1
Name: 4488, dtype: object
------------------
------------------
[(6, 12)]
id                    8036
sid                   8036
sex                      F
cycle                    1
childs                  FD
childbirth               2
dob               8/6/2020
deadwood_days            0
pregancy_days            0
group_id                 9
group_past               8
event                  102
milking_days            59
date             7/29/2022
remark                  13
event_index           4490
ord   

------------------
[(44, 21)]
id                  606021
sid                 606021
sex                      F
cycle                    5
childs                  MA
childbirth               1
dob               6/4/2016
deadwood_days           57
pregancy_days            0
group_id                18
group_past              21
event                  106
milking_days             0
date             7/22/2022
remark                   -
event_index           9103
ord                      1
Name: 9102, dtype: object
------------------
------------------
[(44, 21)]
id                  606021
sid                 606021
sex                      F
cycle                    5
childs                  MA
childbirth               1
dob               6/4/2016
deadwood_days           57
pregancy_days            0
group_id                44
group_past              21
event                  213
milking_days             0
date             7/22/2022
remark             ПОСЛЕД2
event_index           9104
ord 

------------------
[(6, 12)]
id                  811042
sid                 811042
sex                      F
cycle                    2
childs                  FA
childbirth               1
dob              11/6/2018
deadwood_days           66
pregancy_days          230
group_id                 6
group_past               7
event                  303
milking_days           289
date             7/19/2022
remark            F006T012
event_index          10044
ord                      1
Name: 10043, dtype: object
------------------
------------------
[(6, 12)]
id                  811042
sid                 811042
sex                      F
cycle                    2
childs                  FA
childbirth               1
dob              11/6/2018
deadwood_days           66
pregancy_days          230
group_id                12
group_past               7
event                  204
milking_days           289
date             7/19/2022
remark             ХРОМОТА
event_index          10045
ord  

------------------
[(44, 21)]
id                  910033
sid                 910033
sex                      F
cycle                    2
childs                  FA
childbirth               1
dob              10/5/2019
deadwood_days           56
pregancy_days            0
group_id                44
group_past              21
event                  303
milking_days             0
date             7/28/2022
remark            F044T021
event_index          11055
ord                      1
Name: 11054, dtype: object
------------------
------------------
[(44, 21)]
id                  910033
sid                 910033
sex                      F
cycle                    2
childs                  FA
childbirth               1
dob              10/5/2019
deadwood_days           56
pregancy_days            0
group_id                44
group_past              21
event                  206
milking_days             0
date             7/28/2022
remark               ПРОФ2
event_index          11056
ord

,id,sid,sex,cycle,childs,childbirth,dob,deadwood_days,pregancy_days,group_id,group_past,event,milking_days,date,remark,event_index,ord
1965,1001,1001,F,1,FA,1,1/1/2020,0,203,9,9,203,248,7/7/2022,КАН,1966,1
3805,1005,1005,F,1,MA,1,1/1/2020,0,173,6,6,303,254,7/13/2022,F006T007,3806,1
2881,1007,1007,F,1,FA,1,1/1/2020,0,256,5,17,303,268,7/11/2022,F005T006,2882,1
3806,1007,1007,F,1,FA,1,1/1/2020,0,258,6,17,107,275,7/18/2022,214 ДНИ,3807,1
3807,1007,1007,F,1,FA,1,1/1/2020,0,258,6,17,109,278,7/21/2022,CEBA,3808,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11554,912269,912269,F,1,FA,1,12/31/2019,0,223,7,7,303,242,7/19/2022,F007T012,11555,1
11555,912269,912269,F,1,FA,1,12/31/2019,0,223,12,7,204,242,7/19/2022,ХРОМОТА,11556,2
11556,912270,912270,F,1,MA,1,12/31/2019,0,257,7,17,203,275,7/16/2022,КАН,11557,1
11557,912270,912270,F,1,MA,1,12/31/2019,0,257,7,17,107,277,7/18/2022,213 ДНИ,11558,1


**TODO** Спросить что вообще с номерами групп происходит:

In [15]:
df[((df['date'] == '7/29/2022') & (df['id'] == 8036))].head(10)

,id,sid,sex,cycle,childs,childbirth,dob,deadwood_days,pregancy_days,group_id,group_past,event,milking_days,date,remark,event_index,ord
4488,8036,8036,F,1,FD,2,8/6/2020,0,0,6,8,105,59,7/29/2022,614H15481,4489,1
4489,8036,8036,F,1,FD,2,8/6/2020,0,0,9,8,102,59,7/29/2022,13,4490,2
4490,8036,8036,F,1,FD,2,8/6/2020,0,0,6,8,303,59,7/29/2022,F006T012,4491,3
4491,8036,8036,F,1,FD,2,8/6/2020,0,0,12,8,204,59,7/29/2022,ХРОМОТА,4492,4


In [16]:
df[((df['date'] == '7/22/2022') & (df['id'] == 807163))].head(10)

,id,sid,sex,cycle,childs,childbirth,dob,deadwood_days,pregancy_days,group_id,group_past,event,milking_days,date,remark,event_index,ord
9797,807163,807163,F,3,FA,1,7/28/2018,106,0,44,2,303,0,7/22/2022,F044T021,9798,1
9798,807163,807163,F,3,FA,1,7/28/2018,106,0,18,2,106,0,7/22/2022,-,9799,2


На самом деле такое решение имеет смысл только если нет ситуации когда животное кидают из группы в группу по циклу (например перевели из 3 в 12 потом обратно в 3), в таком случае восстановить порядок наверняка невозможно. Но судя по всему такого не происходит

###  Cобытия выздоровления и болезней:
Заболевания КЕТОЗ, МЕТРИТ, ПАРЕЗ, ПОСЛЕД, ПРОФОТ протекают в новотельный период и имеют общий критерий выписки: все послеотельные осложнения животного вылечили, корова пригодна к дальнейшей работе и воспроизводству (примечание при выписке ГИНЕКОЛ). Примечание МОЛОДНЯК к событию ЗДОРОВА имеют только животные 0-й лактации (телки и бычки). Примечание ПРОЧИЕ к событию ЗДОРОВА имеют животные, перенесшие прочие заболевания (событие БОЛЕЗНЬ).

In [42]:
df.head()

,id,sex,cycle,childs,childbirth,deadwood_days,pregancy_days,group_id,group_past,event,...,MAST_КМ6_3,MAST_КМ6_4,MAST_ТМ1_1,MAST_ТМ1_2,MAST_ТМ1_3,MAST_ТМ1_4,MAST_КМ3_1,MAST_КМ3_2,MAST_КМ3_3,MAST_КМ3_4
0,1064,1,1,FA,1,0,0,9,9,BRED,...,0,0,0,0,0,0,0,0,0,0
1,1064,1,1,FA,1,0,0,9,9,TOSCM,...,0,0,0,0,0,0,0,0,0,0
2,1185,1,1,FA,1,0,49,9,43,BRED,...,0,0,0,0,0,0,0,0,0,0
3,1185,1,1,FA,1,0,49,9,43,TOSCM,...,0,0,0,0,0,0,0,0,0,0
4,1260,1,1,FA,1,0,49,5,5,TOSCM,...,0,0,0,0,0,0,0,0,0,0


# Генерация новых фичей